# Example of the SASRec training and inference stages
Note that all the given examples can be run without using PySpark, using only Pandas

In [ ]:
import lightning as L
from lightning.pytorch.loggers import CSVLogger
from lightning.pytorch.callbacks import ModelCheckpoint
from torch.utils.data import DataLoader
import torch

from replay.metrics import OfflineMetrics, Recall, Precision, MAP, NDCG, HitRate, MRR
from replay.splitters import LastNSplitter
from replay.utils import get_spark_session
from replay.data import (
    FeatureHint,
    FeatureInfo,
    FeatureSchema,
    FeatureSource,
    FeatureType,
    Dataset,
)
from replay.models.nn.optimizer_utils import FatOptimizerFactory
from replay.models.nn.sequential.callbacks import (
    ValidationMetricsCallback,
    SparkPredictionCallback,
    PandasPredictionCallback, 
    TorchPredictionCallback
)
from replay.models.nn.sequential.postprocessors import RemoveSeenItems
from replay.data.nn import (
    SequenceTokenizer,
    SequentialDataset,
    TensorFeatureSource,
    TensorSchema,
    TensorFeatureInfo
)
from replay.models.nn.sequential import SasRec
from replay.models.nn.sequential.sasrec import (
    SasPredictionDataset,
    SasTrainingDataset,
    SasValidationDataset,
    SasPredictionBatch
)

import pandas as pd

## Getting a spark session

In [ ]:
spark_session = get_spark_session()

## Prepare data
### Load raw movielens-1M interactions, item features and user features.
In the current implementation, the SASRec does not take into account the features of items or users. They are only used to get a complete list of users and items.

In [ ]:
!pip install rs-datasets

In [4]:
from rs_datasets import MovieLens

In [5]:
movielens = MovieLens("1m")
interactions = movielens.ratings
user_features = movielens.users
item_features = movielens.items

In [6]:
interactions.head()

,user_id,item_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [7]:
user_features.head()

,user_id,gender,age,occupation,zip_code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [8]:
item_features.head()

,item_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


Removing duplicates in the timestamp column without changing the original items order where timestamp is the same

In [9]:
interactions["timestamp"] = interactions["timestamp"].astype("int64")
interactions = interactions.sort_values(by="timestamp")
interactions["timestamp"] = interactions.groupby("user_id").cumcount()
interactions

,user_id,item_id,rating,timestamp
1000138,6040,858,4,0
1000153,6040,2384,4,1
999873,6040,593,5,2
1000007,6040,1961,4,3
1000192,6040,2019,5,4
...,...,...,...,...
825793,4958,2399,1,446
825438,4958,1407,5,447
825724,4958,3264,4,448
825731,4958,2634,3,449


### Split interactions into the train, validation and test datasets using LastNSplitter

In [10]:
raw_test_events, raw_test_gt = LastNSplitter(
    N=1, divide_column="user_id", query_column="user_id", strategy="interactions"
).split(interactions)

raw_validation_events, raw_validation_gt = LastNSplitter(
    N=1, divide_column="user_id", query_column="user_id", strategy="interactions"
).split(raw_test_events)

raw_train_events = raw_validation_events

### Prepare FeatureSchema required to create Dataset

In [15]:
def prepare_feature_schema(is_ground_truth: bool) -> FeatureSchema:
    base_features = FeatureSchema(
        [
            FeatureInfo(
                column="user_id",
                feature_hint=FeatureHint.QUERY_ID,
                feature_type=FeatureType.CATEGORICAL,
            ),
            FeatureInfo(
                column="item_id",
                feature_hint=FeatureHint.ITEM_ID,
                feature_type=FeatureType.CATEGORICAL,
            ),
        ]
    )
    if is_ground_truth:
        return base_features

    all_features = base_features + FeatureSchema(
        [
            FeatureInfo(
                column="timestamp",
                feature_type=FeatureType.NUMERICAL,
                feature_hint=FeatureHint.TIMESTAMP,
            ),
        ]
    )
    return all_features

### Create Dataset for the training stage

In [16]:
train_dataset = Dataset(
    feature_schema=prepare_feature_schema(is_ground_truth=False),
    interactions=raw_train_events,
    query_features=user_features,
    item_features=item_features,
    check_consistency=True,
    categorical_encoded=False,
)

### Create Datasets (events and ground_truth) for the validation stage

In [17]:
validation_dataset = Dataset(
    feature_schema=prepare_feature_schema(is_ground_truth=False),
    interactions=raw_validation_events,
    query_features=user_features,
    item_features=item_features,
    check_consistency=True,
    categorical_encoded=False,
)
validation_gt = Dataset(
    feature_schema=prepare_feature_schema(is_ground_truth=True),
    interactions=raw_validation_gt,
    check_consistency=True,
    categorical_encoded=False,
)

### Create Datasets (events and ground_truth) for the testing stage

In [18]:
test_dataset = Dataset(
    feature_schema=prepare_feature_schema(is_ground_truth=False),
    interactions=raw_test_events,
    query_features=user_features,
    item_features=item_features,
    check_consistency=True,
    categorical_encoded=False,
)
test_gt = Dataset(
    feature_schema=prepare_feature_schema(is_ground_truth=True),
    interactions=raw_test_gt,
    check_consistency=True,
    categorical_encoded=False,
)

### Create the tensor schema
A schema shows the correspondence of columns from the source dataset with the internal representation of tensors inside the model

In [19]:
ITEM_FEATURE_NAME = "item_id_seq"
TIME_FEATURE_NAME = "timestamp_id_seq"

tensor_schema = TensorSchema(
    TensorFeatureInfo(
        name=ITEM_FEATURE_NAME,
        is_seq=True,
        cardinality=train_dataset.item_count,
        feature_type=FeatureType.CATEGORICAL,
        feature_sources=[TensorFeatureSource(FeatureSource.INTERACTIONS, train_dataset.feature_schema.item_id_column)],
        feature_hint=FeatureHint.ITEM_ID,
    )
)

### Create sequential datasets using SequenceTokenizer
The SequentialDataset internally store data in the form of sequences of items sorted by increasing interaction time (timestamp). A SequenceTokenizer is used to convert to this format. In addition, the SequenceTokenizer encodes all categorical columns from the source dataset and stores mapping inside itself.
SequentialDataset.keep_common_query_ids is used to leave only sequences from the same users

In [ ]:
tokenizer = SequenceTokenizer(tensor_schema, allow_collect_to_master=True)
tokenizer.fit(train_dataset)

sequential_train_dataset = tokenizer.transform(train_dataset)

sequential_validation_dataset = tokenizer.transform(validation_dataset)
sequential_validation_gt = tokenizer.transform(validation_gt, [tensor_schema.item_id_feature_name])

sequential_validation_dataset, sequential_validation_gt = SequentialDataset.keep_common_query_ids(
    sequential_validation_dataset, sequential_validation_gt
)

In [ ]:
test_query_ids = test_gt.query_ids
test_query_ids_np = tokenizer.query_id_encoder.transform(test_query_ids)["user_id"].values
sequential_test_dataset = tokenizer.transform(test_dataset).filter_by_query_id(test_query_ids_np)

You can get the user and item mapping and inverse mapping as follows

In [22]:
print(tokenizer.query_id_encoder.mapping, tokenizer.query_id_encoder.inverse_mapping)
print(tokenizer.item_id_encoder.mapping, tokenizer.item_id_encoder.inverse_mapping)

## Train model
### Create SASRec model instance and run the training stage using lightning
After each epoch validation metrics are shown. You can change the list of validation metrics in ValidationMetricsCallback
The model is determined to be the best and is saved if the metric updates its maximum during validation (see the ModelCheckpoint)

In [ ]:
MAX_SEQ_LEN = 200
BATCH_SIZE = 512
NUM_WORKERS = 4

model = SasRec(
    tensor_schema,
    block_count=2,
    head_count=2,
    max_seq_len=MAX_SEQ_LEN,
    embedding_dim=300,
    dropout_rate=0.5,
    optimizer_factory=FatOptimizerFactory(learning_rate=0.001),
)
checkpoint_callback = ModelCheckpoint(
    dirpath=".checkpoints",
    save_top_k=1,
    verbose=True,
    monitor="recall@10",
    mode="max",
)

validation_metrics_callback = ValidationMetricsCallback(
    metrics=["map", "ndcg", "recall"],
    ks=[1, 5, 10, 20],
    item_count=train_dataset.item_count,
    postprocessors=[RemoveSeenItems(sequential_validation_dataset)]
)

csv_logger = CSVLogger(save_dir=".logs/train", name="SASRec_example")

trainer = L.Trainer(
    max_epochs=100,
    callbacks=[checkpoint_callback, validation_metrics_callback],
    logger=csv_logger,
)

train_dataloader = DataLoader(
    dataset=SasTrainingDataset(
        sequential_train_dataset,
        max_sequence_length=MAX_SEQ_LEN,
    ),
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=4,
    pin_memory=True,
)

validation_dataloader = DataLoader(
    dataset=SasValidationDataset(
        sequential_validation_dataset,
        sequential_validation_gt,
        sequential_train_dataset,
        max_sequence_length=MAX_SEQ_LEN,
    ),
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    pin_memory=True,
)

trainer.fit(
    model,
    train_dataloaders=train_dataloader,
    val_dataloaders=validation_dataloader,
)

The path to the best model is saved inside checkpoint_callback

In [24]:
best_model = SasRec.load_from_checkpoint(checkpoint_callback.best_model_path)

## Inference stage
### Prepare Dataloader and logger

In [25]:
prediction_dataloader = DataLoader(
    dataset=SasPredictionDataset(
        sequential_test_dataset,
        max_sequence_length=MAX_SEQ_LEN,
    ),
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    pin_memory=True,
)

csv_logger = CSVLogger(save_dir=".logs/test", name="SASRec_example")

### Run inference
You can get the recommendations in three formats: PySpark DataFrame, Pandas DataFrame, PyTorch tensors. Each of the types corresponds a callback
You can filter the results using postprocessors strategy. For example the RemoveSeenItems postprocessor is filtering out the items that already have been seen in test dataset
You don't need to use all three callbacks. This is shown only for example

In [ ]:
TOPK = [1, 10, 20, 100]

postprocessors = [RemoveSeenItems(sequential_test_dataset)]

spark_prediction_callback = SparkPredictionCallback(
    spark_session=spark_session,
    top_k=max(TOPK),
    query_column="user_id",
    item_column="item_id",
    rating_column="score",
    postprocessors=postprocessors,
)

pandas_prediction_callback = PandasPredictionCallback(
    top_k=max(TOPK),
    query_column="user_id",
    item_column="item_id",
    rating_column="score",
    postprocessors=postprocessors,
)

torch_prediction_callback = TorchPredictionCallback(
    top_k=max(TOPK),
    postprocessors=postprocessors,
)


trainer = L.Trainer(
    callbacks=[spark_prediction_callback, pandas_prediction_callback, torch_prediction_callback], 
    logger=csv_logger, 
    inference_mode=True
)
trainer.predict(model, dataloaders=prediction_dataloader)

spark_res = spark_prediction_callback.get_result()
pandas_res = pandas_prediction_callback.get_result()
torch_user_ids, torch_item_ids, torch_scores = torch_prediction_callback.get_result()

In [27]:
spark_res.show()

+-------+-------+------------------+
|user_id|item_id|             score|
+-------+-------+------------------+
|      0|   1642| 8.845483779907227|
|      0|    360| 8.334686279296875|
|      0|   2012| 8.094024658203125|
|      0|     47| 8.042309761047363|
|      0|   2325| 8.015921592712402|
|      0|   1019| 7.517959117889404|
|      0|   2020| 7.322227954864502|
|      0|   2027| 7.232193946838379|
|      0|   1020| 7.163656711578369|
|      0|   2014|  7.16225528717041|
|      0|    612| 7.088329792022705|
|      0|   2018| 7.042888164520264|
|      0|   2016|  7.01570463180542|
|      0|   2009| 7.010195255279541|
|      0|     12| 6.953461647033691|
|      0|   3090| 6.774460792541504|
|      0|   1012| 6.583562850952148|
|      0|   2072| 6.522708415985107|
|      0|   2965|6.4962921142578125|
|      0|   1382| 6.485069751739502|
+-------+-------+------------------+
only showing top 20 rows



In [28]:
pandas_res

,user_id,item_id,score
0,0,1642,8.845484
0,0,360,8.334686
0,0,2012,8.094025
0,0,47,8.04231
0,0,2325,8.015922
...,...,...,...
6039,6039,2890,3.084756
6039,6039,1734,3.080646
6039,6039,413,3.071112
6039,6039,467,3.059183


In [29]:
print(torch_user_ids[0], torch_item_ids[0], torch_scores[0])

tensor(0) tensor([1642,  360, 2012,   47, 2325, 1019, 2020, 2027, 1020, 2014,  612, 2018,
        2016, 2009,   12, 3090, 1012, 2072, 2965, 1382,  547,  700, 1050, 2021,
        2073, 3088,  667, 2068,  105, 2011, 1017, 2023,  310, 2807, 1010, 1979,
        2030, 1011, 2315,  974, 1964, 3546, 1850,  582, 2918, 1990, 2070, 2830,
        1346, 2731, 1459, 2033, 1743,  314,  259, 1018, 2285,  358,  592,   33,
        1262, 1058, 2985, 1977, 2631, 1851, 3144, 2047, 3218, 1002, 3414, 3327,
         236, 2013,  554, 3676, 1004, 3328,  476, 2092, 2015,    1, 1547,   59,
         791, 2640, 2692,  999, 2788, 1936, 3329, 1072, 1664, 1000, 1539, 3106,
        3690, 2677, 3682,  826]) tensor([8.8455, 8.3347, 8.0940, 8.0423, 8.0159, 7.5180, 7.3222, 7.2322, 7.1637,
        7.1623, 7.0883, 7.0429, 7.0157, 7.0102, 6.9535, 6.7745, 6.5836, 6.5227,
        6.4963, 6.4851, 6.4677, 6.4470, 6.3658, 6.3478, 6.2661, 6.2315, 6.2047,
        6.1852, 6.1586, 6.1142, 6.1109, 6.1086, 6.0757, 6.0644, 5.9111, 5.905

Suppose we want to get the recomendations in PySpark format. 
Let's get the inverse representation of labels using inverse_transform method.

Note that the reverse representation can only be obtained for PySpark and Pandas formats. When working with PyTorch tensors, the reverse representation must be done manually

In [30]:
recommendations = tokenizer.query_and_item_id_encoder.inverse_transform(spark_res)

In [31]:
recommendations.show()

+------------------+-------+-------+
|             score|user_id|item_id|
+------------------+-------+-------+
| 8.845483779907227|      1|   1688|
| 8.334686279296875|      1|    364|
| 8.094024658203125|      1|   2081|
| 8.042309761047363|      1|     48|
| 8.015921592712402|      1|   2394|
| 7.517959117889404|      1|   1032|
| 7.322227954864502|      1|   2089|
| 7.232193946838379|      1|   2096|
| 7.163656711578369|      1|   1033|
|  7.16225528717041|      1|   2083|
| 7.088329792022705|      1|    616|
| 7.042888164520264|      1|   2087|
|  7.01570463180542|      1|   2085|
| 7.010195255279541|      1|   2078|
| 6.953461647033691|      1|     13|
| 6.774460792541504|      1|   3159|
| 6.583562850952148|      1|   1025|
| 6.522708415985107|      1|   2141|
|6.4962921142578125|      1|   3034|
| 6.485069751739502|      1|   1405|
+------------------+-------+-------+
only showing top 20 rows



### Calculating metrics

In [32]:
init_args = {"query_column": "user_id", "rating_column": "score"}

In [33]:
result_metrics = OfflineMetrics(
    [Recall(TOPK), Precision(TOPK), MAP(TOPK), NDCG(TOPK), MRR(TOPK), HitRate(TOPK)], **init_args
)(recommendations.toPandas(), raw_test_gt)

In [34]:
result_metrics

{'Recall@1': 0.06655629139072848,
 'Recall@10': 0.28725165562913907,
 'Recall@20': 0.3990066225165563,
 'Recall@100': 0.676158940397351,
 'Precision@1': 0.06655629139072848,
 'Precision@10': 0.028725165562913907,
 'Precision@20': 0.019950331125827815,
 'Precision@100': 0.00676158940397351,
 'MAP@1': 0.06655629139072848,
 'MAP@10': 0.12269020025228634,
 'MAP@20': 0.13042627266652343,
 'MAP@100': 0.13730766168488456,
 'NDCG@1': 0.06655629139072848,
 'NDCG@10': 0.16109587380537335,
 'NDCG@20': 0.18932812461033316,
 'NDCG@100': 0.2402926417265344,
 'MRR@1': 0.06655629139072848,
 'MRR@10': 0.12269020025228634,
 'MRR@20': 0.13042627266652343,
 'MRR@100': 0.13730766168488456,
 'HitRate@1': 0.06655629139072848,
 'HitRate@10': 0.28725165562913907,
 'HitRate@20': 0.3990066225165563,
 'HitRate@100': 0.676158940397351}

## Example of launching an inference for a single user without using a trainer (in order to speed up)
An example for the production of an online script

Let's assume that the user's sequence consisted of a sequence of items [1, 2, 3, 4, 5]. 
Сreate a padding mask corresponding to the sequence of items.

It is important to take only the latest MAX_SEQ_LEN or less items.

In [35]:
item_sequence = torch.Tensor([1, 2, 3, 4, 5]).unsqueeze(0)
padding_mask = torch.ones_like(item_sequence[:][-MAX_SEQ_LEN:], dtype=torch.bool)
sequence_item_count = item_sequence.shape[1]

If the size is less than MAX_SEQ_LEN, then we pad the tensors with zeros on the left

In [36]:
item_sequence = torch.nn.functional.pad(
    item_sequence, (MAX_SEQ_LEN - sequence_item_count, 0), value=0
)
padding_mask = torch.nn.functional.pad(
    padding_mask, (MAX_SEQ_LEN - sequence_item_count, 0), value=0
)

### Wrapping created tensors in the SASRecPredictionBatch entity

In [37]:
batch = SasPredictionBatch(
    query_id=torch.arange(0, item_sequence.shape[0], 1).long(),
    padding_mask=padding_mask.bool(),
    features={ITEM_FEATURE_NAME: item_sequence.long()}
)

### Run predict step of the SASRec and get scores from the model

In [38]:
with torch.no_grad():
    scores = model.predict_step(batch, 0)
scores

tensor([[ 2.9611,  0.3749,  0.0170,  ..., -1.6915, -1.7273,  1.6130]])

### Getting three items with the highest score

In [39]:
torch.topk(scores, k=3).indices

tensor([[   5,  476, 2502]])